In [1]:
from reachy_sdk import ReachySDK

In [2]:
reachy = ReachySDK('localhost')

Check if you see the five joints of the head.

In [3]:
reachy.head

<Head joints=<Holder
	<Joint name="l_antenna" pos="-0.44" mode="compliant">
	<Joint name="r_antenna" pos="-1.32" mode="compliant">
	<Joint name="neck_disk_top" pos="-59.39" mode="compliant">
	<Joint name="neck_disk_middle" pos="-64.42" mode="compliant">
	<Joint name="neck_disk_bottom" pos="-56.34" mode="compliant">
>>

Use look_at to see if the zero has been actually done.

In [4]:
reachy.turn_on('head')

In [5]:
reachy.head.look_at(x=0.5, y=0, z=0, duration=1.0)

Reproduce the look_at sequence of the [documentation](https://pollen-robotics.github.io/reachy-2021-docs/sdk/first-moves/head/#orbita-look_at-method).

In [6]:
import time

look_right = reachy.head.look_at(x=0.5, y=-0.5, z=0.1, duration=1.0)
time.sleep(0.1)
look_down = reachy.head.look_at(x=0.5, y=0, z=-0.4, duration=1.0)
time.sleep(0.1)
look_left = reachy.head.look_at(x=0.5, y=0.3, z=-0.3, duration=1.0)
time.sleep(0.1)
look_front = reachy.head.look_at(x=0.5, y=0, z=0, duration=1.0)

Make the head follow the end-effector. It is a good opportunity to check the forward kinematics of each arm as well.

With the right arm:

In [8]:
try:
    x, y, z = reachy.r_arm.forward_kinematics()[:3,-1] # We want the translation part of Reachy's pose matrix
    reachy.head.look_at(x=x, y=y, z=z-0.05, duration=1.0) # There is a 5cm offset on the z axis

    time.sleep(0.5)

    while True:
        x, y, z = reachy.r_arm.forward_kinematics()[:3,-1]
        gp_dic = reachy.head._look_at(x, y, z - 0.05)
        reachy.head.neck_disk_bottom.goal_position = gp_dic[reachy.head.neck_disk_bottom]
        reachy.head.neck_disk_middle.goal_position = gp_dic[reachy.head.neck_disk_middle]
        reachy.head.neck_disk_top.goal_position = gp_dic[reachy.head.neck_disk_top]
        time.sleep(0.01)
except AttributeError:
    print('Reachy has no right arm!')

With the left arm:

In [11]:
try:
    x, y, z = reachy.l_arm.forward_kinematics()[:3,-1] # We want the translation part of Reachy's pose matrix
    reachy.head.look_at(x=x, y=y, z=z-0.05, duration=1.0) # There is a 5cm offset on the z axis

    time.sleep(0.5)

    while True:
        x, y, z = reachy.l_arm.forward_kinematics()[:3,-1]
        gp_dic = reachy.head._look_at(x, y, z - 0.05)
        reachy.head.neck_disk_bottom.goal_position = gp_dic[reachy.head.neck_disk_bottom]
        reachy.head.neck_disk_middle.goal_position = gp_dic[reachy.head.neck_disk_middle]
        reachy.head.neck_disk_top.goal_position = gp_dic[reachy.head.neck_disk_top]
        time.sleep(0.01)
except AttributeError:
    print('Reachy has no left arm!')

Check the antennas.

First check that the l_antenna is actually the left and that r_antenna is the right.

In [13]:
reachy.head.l_antenna.goal_position = -40

In [14]:
reachy.head.l_antenna.goal_position = 0

In [15]:
reachy.head.r_antenna.goal_position = -40

In [16]:
reachy.head.r_antenna.goal_position = 0

Reproduce the sad and happy movements from the [documentation](https://pollen-robotics.github.io/reachy-2021-docs/sdk/first-moves/head/#antennas).

In [17]:
def happy_antennas():
    reachy.head.l_antenna.speed_limit = 0.0
    reachy.head.r_antenna.speed_limit = 0.0
    
    for _ in range(9):
        reachy.head.l_antenna.goal_position = 10.0
        reachy.head.r_antenna.goal_position = -10.0

        time.sleep(0.1)

        reachy.head.l_antenna.goal_position = -10.0
        reachy.head.r_antenna.goal_position = 10.0

        time.sleep(0.1)
    
    reachy.head.l_antenna.goal_position = 0.0
    reachy.head.r_antenna.goal_position = 0.0
        
def sad_antennas():
    reachy.head.l_antenna.speed_limit = 70.0
    reachy.head.r_antenna.speed_limit = 70.0
    
    reachy.head.l_antenna.goal_position = 140.0
    reachy.head.r_antenna.goal_position = -140.0
    
    time.sleep(5.0)
    
    reachy.head.l_antenna.goal_position = 0.0
    reachy.head.r_antenna.goal_position = 0.0

In [18]:
happy_antennas()

In [19]:
sad_antennas()